# Module 3.1: Continuum Logic (The Artificial Wall)

**The Concept:** In standard computer science, we are taught that Classical Computing (0/1) and Quantum Computing (Vectors) are entirely different worlds.

This is false. The "Wall" is just a habit of using integers.

If we treat a bit not as an integer, but as a continuous vector on a circle, we can perform "Classical" logic using "Quantum" math (Linear Algebra and Rotation). 

*   **Classical 0:** 0 degrees
*   **Classical 1:** 90 degrees ($\pi/2$)
*   **Quantum Superposition:** Anything in between.

Here, we build a **Differentiable Gate** that learns to be a NOT gate or a Hadamard gate simply by sliding its rotation angle.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class ContinuumBit:
    """
    Represents a bit not as '0' or '1', but as a coordinate on a
    unit circle (a normalized vector).
    """
    def __init__(self, state_0_prob=1.0):
        # We model the state as an angle (theta).
        # 0.0 radians = State 0 ([1, 0])
        # pi/2 radians = State 1 ([0, 1])
        self.theta = np.arccos(np.sqrt(state_0_prob))

    def get_state_vector(self):
        # Convert angle back to probability amplitudes
        # This is pure geometry.
        alpha = np.cos(self.theta) # Component for 0
        beta = np.sin(self.theta)  # Component for 1
        return np.array([alpha, beta])

    def measure(self):
        """
        The 'Collapse'.
        Samples the distribution to give a concrete 0 or 1.
        """
        vector = self.get_state_vector()
        prob_1 = vector[1] ** 2 # Born Rule (Probability = Amplitude^2)
        return 1 if np.random.rand() < prob_1 else 0

class DifferentiableGate:
    """
    Instead of a hardcoded logic gate, this is a parametric gate.
    It has a 'weight' (angle) that we can adjust smoothly.
    """
    def __init__(self, initial_rotation=0.0):
        self.param = initial_rotation # The "Angle" of operation

    def forward(self, bit):
        # Logic is just movement in the state space (Rotation)
        bit.theta += self.param
        return bit

    def backward(self, error, learning_rate=0.1):
        # Differentiable Programming: Adjust logic based on error
        # This allows the software to 'learn' its own logic.
        self.param -= error * learning_rate

In [ ]:
# --- SIMULATION LOOP ---

def run_learning_experiment(target_prob, iterations=50, label="Unknown"):
    print(f"\n--- Experiment: Teaching Gate to target Probability {target_prob} ({label}) ---")

    # Initialize with random logic (random rotation)
    my_gate = DifferentiableGate(initial_rotation=np.random.uniform(-1, 1))

    losses = []

    for i in range(iterations):
        # 1. Prepare fresh bit at State 0
        bit = ContinuumBit(state_0_prob=1.0)

        # 2. Process (Rotate)
        my_gate.forward(bit)

        # 3. Compute Current Probability of 1
        current_vector = bit.get_state_vector()
        current_prob_1 = current_vector[1] ** 2

        # 4. Compute Loss (Error)
        # We want the probability to match the target
        loss = current_prob_1 - target_prob
        losses.append(abs(loss))

        # 5. Learn (Backpropagate)
        # If probability is too high, rotate back. Too low, rotate forward.
        my_gate.backward(loss, learning_rate=0.2)

        if i % 10 == 0:
            print(f"Iter {i}: Angle = {np.degrees(my_gate.param):.2f}° | Prob = {current_prob_1:.4f}")

    print(f"Final Result: The gate learned a rotation of {np.degrees(my_gate.param):.2f}°")
    return losses

In [ ]:
# --- EXECUTION ---

# 1. Teach it to be a NOT Gate (Turn 0 -> 1)
# Target Probability = 1.0 (90 degrees)
loss_not = run_learning_experiment(1.0, label="NOT Gate")

# 2. Teach it to be a HADAMARD Gate (Turn 0 -> 0.5 Superposition)
# Target Probability = 0.5 (45 degrees)
loss_hadamard = run_learning_experiment(0.5, label="Superposition/Hadamard")

# --- VISUALIZATION ---
plt.figure(figsize=(10, 5))
plt.plot(loss_not, label='Learning NOT (Target 1.0)', linewidth=2)
plt.plot(loss_hadamard, label='Learning Hadamard (Target 0.5)', linewidth=2)
plt.title("Differentiable Logic: Minimizing Error over Time")
plt.xlabel("Iterations")
plt.ylabel("Error (Loss)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()